In [1]:
import xarray as xr
import numpy as np
import os
import arrow
import pandas as pd
import metpy.calc as mpcalc
from metpy.units import units
import glob
import re

In [2]:
if os.environ['COMPUTERNAME'] == 'DESKTOP-EQAO3M5':
  computer_flag = 'home'
else:
  computer_flag = 'office'

if computer_flag == 'home':
  file_dir = "F:/github/pythonScript/seafog/"
else:
  file_dir = "H:/github/python/seafog/"
file_station = os.path.normpath(os.path.join(file_dir, f'./data/station/all_stations_v2.csv'))

In [14]:
df_station = pd.read_csv(file_station,sep=',',na_values=[9999], parse_dates=['index', 'DDATETIME'],index_col="index")
df_station

,DDATETIME,V20059,V13019,V13003
index,,,,
2013-01-31 04:00:00,2013-01-31 04:00:00,12756.0,0.0,70.0
2013-01-31 05:00:00,2013-01-31 05:00:00,22272.0,0.0,64.0
2013-01-31 06:00:00,2013-01-31 06:00:00,24547.0,0.0,66.0
2013-01-31 07:00:00,2013-01-31 07:00:00,15263.0,0.0,66.0
2013-01-31 08:00:00,2013-01-31 08:00:00,18838.0,0.0,74.0
...,...,...,...,...
2022-10-29 20:00:00,2022-10-29 20:00:00,6622.0,0.0,83.0
2022-10-29 21:00:00,2022-10-29 21:00:00,9435.0,0.0,84.0
2022-10-29 22:00:00,2022-10-29 22:00:00,9630.0,0.0,78.0


In [6]:
# df_station.insert(loc=2, column='Pre24VisMin', value=np.nan)

In [27]:
iTime = df_station.iloc[0]

In [34]:
iTime = df_station.iloc[43]['DDATETIME']
preTime = iTime + pd.Timedelta(-24,unit='h')
df_station.loc[(df_station.index>preTime) & (df_station.index<=iTime), 'V20059'].min()

199.0

In [35]:
def getVisMin24(time):
    iTime = df_station.loc[time, 'DDATETIME']
    preTime = iTime + pd.Timedelta(-24,unit='h')
    minVis = df_station.loc[(df_station.index>preTime) & (df_station.index<=iTime), 'V20059'].min()
    return minVis

In [36]:
preVisMin = df_station['DDATETIME'].apply(getVisMin24)


In [38]:
df_station['Pre24VisMin'] = preVisMin
print(df_station)

                              DDATETIME   V20059  V13019  V13003  Pre24VisMin
index                                                                        
2013-01-31 04:00:00 2013-01-31 04:00:00  12756.0     0.0    70.0      12756.0
2013-01-31 05:00:00 2013-01-31 05:00:00  22272.0     0.0    64.0      12756.0
2013-01-31 06:00:00 2013-01-31 06:00:00  24547.0     0.0    66.0      12756.0
2013-01-31 07:00:00 2013-01-31 07:00:00  15263.0     0.0    66.0      12756.0
2013-01-31 08:00:00 2013-01-31 08:00:00  18838.0     0.0    74.0      12756.0
...                                 ...      ...     ...     ...          ...
2022-10-29 20:00:00 2022-10-29 20:00:00   6622.0     0.0    83.0       3795.0
2022-10-29 21:00:00 2022-10-29 21:00:00   9435.0     0.0    84.0       3795.0
2022-10-29 22:00:00 2022-10-29 22:00:00   9630.0     0.0    78.0       4324.0
2022-10-29 23:00:00 2022-10-29 23:00:00  16545.0     0.0    78.0       5453.0
2022-10-30 00:00:00 2022-10-30 00:00:00  17840.0     0.0    76.0

In [19]:
df_station.loc[iTime, 'V20059']

12089.0

In [16]:
df_station.index

DatetimeIndex(['2013-01-31 04:00:00', '2013-01-31 05:00:00',
               '2013-01-31 06:00:00', '2013-01-31 07:00:00',
               '2013-01-31 08:00:00', '2013-01-31 09:00:00',
               '2013-01-31 10:00:00', '2013-01-31 11:00:00',
               '2013-01-31 12:00:00', '2013-01-31 13:00:00',
               ...
               '2022-10-29 15:00:00', '2022-10-29 16:00:00',
               '2022-10-29 17:00:00', '2022-10-29 18:00:00',
               '2022-10-29 19:00:00', '2022-10-29 20:00:00',
               '2022-10-29 21:00:00', '2022-10-29 22:00:00',
               '2022-10-29 23:00:00', '2022-10-30 00:00:00'],
              dtype='datetime64[ns]', name='index', length=83381, freq=None)